
**رگرسیون لجستیک**


https://aparat.com/v/YzsBF

در این مدل از  تحلیل مولفه های اصلی استفاده می کند تا بعد مولفه های عامل (مستقل) کاهش  پیدا کند سپس مدل رگرسیون لجستیک اجرا می شود

در این مدول  یک پایپ لاین و گریدسرچ هست 

  pipeline و GridSearchCV 
مسئله یافتن بهترین تعدادبهینه مولفه در مدل است

*در ابتدا ماژول را فراخوانی کنید!*


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn import datasets
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

در ابتدای کار باید تعداد مولفه های اصلی ( عامل/ متغیر مستقل) کاهش بیاید
درابتدا باید یک مدل تعریف کنید 
در اینجا  تحلیل مولفه اصلی  و لجستیک رگرسیون  تعریف شود.

In [ ]:
# Define a pipeline to search for the best combination of PCA truncation
# and classifier regularization.
pca = PCA()
# set the tolerance to a large value to make the example faster
logistic = LogisticRegression(max_iter=10000, tol=0.1)

مراحل کاری را تعریف کنید
در ابتدا تحلیل مولفه اصلی  اجرا شود بعد مدل رگرسیون

In [ ]:
pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

**داده های مثال**

دراینجا متغیر X = 
متغیر مستقل است


In [ ]:
X_digits, y_digits = datasets.load_digits(return_X_y=True)

نمایش تعداد عامل  64  عدد است
تعداد نمونه های ردیف هم 1797 عدد است 

In [ ]:
print("Shape: ",X_digits.shape)
#نمونه اول
X_digits[0]

پارامترها تعریف شود با گذاشتن  علامت 
__
پارامترهای  که بهینه شود

  در اینجا **تعداد مولفه  اصلی** با تعداد تعیین شده و  

**C** دررگرسیون لجستیک
 است.

  دامنه آن از منفی 4 تا مثبت 4 است

In [ ]:
# Parameters of pipelines can be set using ‘__’ separated parameter names:
param_grid = {
    'pca__n_components': [5, 15, 30, 45, 64],
    'logistic__C': np.logspace(-4, 4, 4),
}

***وظیفه را برای جستجو پیدا کنید***

In [ ]:
search = GridSearchCV(pipe, param_grid, n_jobs=-1)
search.fit(X_digits, y_digits)



>  نتیجه را مشاهده نمایید



In [ ]:
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)

*نمایش نمودار تغییرات*

In [ ]:
# Plot the PCA spectrum
pca.fit(X_digits)

fig, (ax0, ax1) = plt.subplots(nrows=2, sharex=True, figsize=(6, 6))
ax0.plot(np.arange(1, pca.n_components_ + 1),
         pca.explained_variance_ratio_, '+', linewidth=2)
ax0.set_ylabel('PCA explained variance ratio')

ax0.axvline(search.best_estimator_.named_steps['pca'].n_components,
            linestyle=':', label='n_components chosen')
ax0.legend(prop=dict(size=12))

#
# For each number of components, find the best classifier results
results = pd.DataFrame(search.cv_results_)
components_col = 'param_pca__n_components'
best_clfs = results.groupby(components_col).apply(
    lambda g: g.nlargest(1, 'mean_test_score'))

best_clfs.plot(x=components_col, y='mean_test_score', yerr='std_test_score',
               legend=False, ax=ax1)
ax1.set_ylabel('Classification accuracy (val)')
ax1.set_xlabel('n_components')

plt.xlim(-1, 70)

plt.tight_layout()
plt.show()